In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class SymbolicLayer:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.fc = nn.Linear(input_size, output_size)
        self.W = torch.randn(input_size, output_size)

    def forward(self, x):
        for f, x in
            x = torch.tensor([x])
            output_2.append(f(x))

In [15]:
class SymbolicNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, functions: list):
        super(SymbolicNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.functions = functions
        self.fc1 = torch.nn.Linear(input_size, hidden_size)

    def forward(self, x):
        output_1 = self.fc1(x)
        output_2 = []
        for f, x in zip(self.functions, output_1):
            x = torch.tensor([x])
            output_2.append(f(x))
        output = torch.sum(torch.tensor(output_2))
        return output
    
    def train(self, samples, true, epochs=100, lr=0.01):
        optimizer = optim.SGD(self.parameters(), lr=lr)
        loss_fn = nn.MSELoss()
        for epoch in range(epochs):
            for x, y in zip(samples, true):
                x = torch.tensor([x])
                y = torch.tensor([y])
                optimizer.zero_grad()
                output = self.forward(x)
                loss = loss_fn(output, y)
                loss.backward()
                optimizer.step()
                if epoch % 10 == 0:
                    print(f'Epoch: {epoch} Loss: {loss.item()}')

In [16]:
functions = [torch.square, torch.absolute, torch.sin, torch.cos]

samples = torch.randn(1, 100)
output = torch.square(samples)

In [17]:
model = SymbolicNet(1, 4, 1, functions)

In [18]:
model(torch.tensor([2.0]))

tensor(2.2082)

In [19]:
model.train(x, y)

/home/emil/anaconda3/envs/DeepSymRegTorch/lib/python3.10/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn